<a href="https://colab.research.google.com/github/webbigdata-jp/python_sample/blob/main/ALMA_7B_Ja_GPTQ_Ja_En_Free_Colab_sample.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Original Model ALMA-7B.  (26.95GB)  
https://huggingface.co/haoranxu/ALMA-7B  

ALMA-7B-Ja is a machine translation model that uses ALMA's learning method to translate Japanese to English and English to Japanese.(13.3GB)  
https://huggingface.co/webbigdata/ALMA-7B-Ja  

ALMA-7B-Ja-GPTQ-Ja-En-Free is a quantized(miniaturized) version of the ALMA-7B-Ja model.(3.9GB)  
https://huggingface.co/webbigdata/ALMA-7B-Ja-GPTQ-Ja-En

In [3]:
%%bash
pip install auto-gptq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.1/258.1 kB 15.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 20.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 25.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 43.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.6/85.6 kB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 66.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 17.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 26.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.7 MB/s eta 0:00:00


In [4]:
import torch
from transformers import AutoTokenizer
from auto_gptq import AutoGPTQForCausalLM

quantized_model_dir = "webbigdata/ALMA-7B-Ja-GPTQ-Ja-En"
model_basename = "gptq_model-4bit-128g"

tokenizer = AutoTokenizer.from_pretrained(quantized_model_dir)

model = AutoGPTQForCausalLM.from_quantized(
        quantized_model_dir,
        model_basename=model_basename,
        use_safetensors=True,
        device="cuda:0")

prompt1="Translate this from Japanese to English:\nJapanese: 量子化するとモデルの性能はどのくらい劣化してしまうのでしょうか？\nEnglish:"
input_ids = tokenizer(prompt1, return_tensors="pt", padding=True, max_length=200, truncation=True).input_ids.cuda()

# Translation
with torch.no_grad():
    generated_ids = model.generate(input_ids=input_ids, num_beams=5, max_new_tokens=250, do_sample=True, temperature=0.6, top_p=0.9)
outputs = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)
print(outputs)

prompt2="Translate this from English to Japanese:\nEnglish: I hope that this quantization model will be useful to people.?\nJapanese:"
input_ids = tokenizer(prompt2, return_tensors="pt", padding=True, max_length=200, truncation=True).input_ids.cuda()
with torch.no_grad():
    generated_ids = model.generate(input_ids=input_ids, num_beams=5, max_new_tokens=250, do_sample=True, temperature=0.6, top_p=0.9)
outputs = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)
print(outputs)



Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


['Translate this from Japanese to English:\nJapanese: 量子化するとモデルの性能はどのくらい劣化してしまうのでしょうか？\nEnglish:How much does the performance of the model degrade when it is quantized?']
['Translate this from English to Japanese:\nEnglish: I hope that this quantization model will be useful to people.?\nJapanese:この量子モデルが人々の役に立つことを望んでいます。']
